# The Hangman Game

## Instruction:
For this project, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [3]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
np.set_printoptions(threshold=np.inf)
from copy import deepcopy
import random
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

Using TensorFlow backend.


In [4]:
HANGMAN_URL = ""
thre = 0.8
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None,batch_size=1000,epochs=5,test_size=0.33):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.max_length = 0
        self.test_size = test_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.train_model()  # five-layer bidirectional LSTM

        
        
    
            
    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        
        # clean the word so that we strip away the space characters
        clean_word = word[::2]
        
        # _abc -> 27 1 2 3
        encoded_word = self.encode_word(clean_word)
        
        # _abc -> 27 1 2 3 0 0 0 0 ...
        paded_word = pad_sequences([encoded_word], maxlen=self.max_length, padding='post')
                
        # predict and decode, get [Pa Pb Pc ...Pz]
        pred = self.model.predict(paded_word) 
        
        # [Pa Pb Pc ...Pz] -> argmax P index list of such as [b s a d f ....]
        pred = self.get_order(pred[0])
        
        for letter in pred:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break         
        return guess_letter
    
    def encode_word(self, word):# _abc -> 27 1 2 3
        encoded_word = []
        for cha in word:
            if ord(cha) == ord("_"):
                encoded_word.append(27)
            else:
                encoded_word.append(ord(cha)- ord("a")+1)
        return encoded_word

    def get_xy(self):
        encoded_dictionary = []
        for i in self.full_dictionary:
            encoded_dictionary.append(self.encode_word(i))
        X = []
        y = []
        for encoded_word in encoded_dictionary:
            dic = {cha:[] for cha in list(set(encoded_word))}
            for i in range(len(encoded_word)):
                dic[encoded_word[i]].append(i)
            cha_lis = list(dic.keys())
            for cha,lis in dic.items():
                masked_word = deepcopy(encoded_word)
                for pos in lis:
                    masked_word[pos] = 27
#                 print(masked_word)
                target = cha -1 
                X.append(masked_word + [i]+[i])
                y.append(target)
                times = 0
                seen = [cha]
                new_masked_word = deepcopy(masked_word)
                while times < len(cha_lis):
                    j = random.randint(0,len(cha_lis)-1)   
                    times+=1
                    if cha_lis[j] in seen:
                        continue
                    seen.append(cha_lis[j])
                    for pos in dic[cha_lis[j]]:
                        new_masked_word[pos] = 27
                    X.append(new_masked_word+ [i]+[j])
#                     print(new_masked_word)
                    y.append(target)
                    
        print("done!")
        
        for i in range(len(X)):
            X[i] = X[i][:-2]
            self.max_length = max(self.max_length, len(X[i]))
        print(X[:100])
        print(y[:100])
        # pad documents to a max length
        X = pad_sequences(X, maxlen=self.max_length, padding='post')
        y = to_categorical(y, num_classes=26)
        train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=self.test_size, random_state=42)
        return (train_x, test_x, train_y, test_y)

    def train_model(self): # five-layer bidirectional LSTM
        (train_x, test_x, train_y, test_y) = self.get_xy()

        # define the model
        self.model = Sequential()
        self.model.add(Embedding(28, 128, input_length=self.max_length))
        self.model.add(Bidirectional(LSTM(64, return_sequences=True)))
        self.model.add(Bidirectional(LSTM(64, return_sequences=True)))
        self.model.add(Bidirectional(LSTM(64, return_sequences=True)))
        self.model.add(Bidirectional(LSTM(64, return_sequences=True)))
        self.model.add(Bidirectional(LSTM(64)))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(26, activation='softmax'))

        # compile the model
        self.model.compile(optimizer="adam",loss = "categorical_crossentropy", metrics=['accuracy', 'top_k_categorical_accuracy'])

        # summarize the model
        print(self.model.summary())

        # fit and evaluate the model
        self.model.fit(train_x, train_y,
                batch_size=self.batch_size,
                epochs=self.epochs,
                validation_data=[test_x, test_y])
        
        return
    
    def get_order(self, prediction): # [Pa Pb Pc ...Pz] -> argmax P index list of such as [b s a d f ....]
        dic = {i:prediction[i] for i in range(len(prediction))}
        sorted_cha_list = [chr(index + ord("a")) for index, value in sorted(dic.items(), key=lambda x:x[1],reverse = True)]
        return sorted_cha_list
    
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        try:
            # response = self.session.request(
            response = requests.request(
                method or "GET",
                HANGMAN_URL + path,
                timeout=self.timeout,
                params=args,
                data=post_args)
        except requests.HTTPError as e:
            response = json.loads(e.read())
            raise HangmanAPIError(response)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [8]:
api = HangmanAPI(access_token="", timeout=2000, batch_size=3000,epochs=10,test_size=0.1)

done!
[[27, 27, 27], [27, 27, 27, 27, 27, 27], [27, 27, 27, 19], [1, 1, 1, 27], [27, 27, 3, 8, 5, 14], [27, 27, 3, 8, 5, 27], [27, 27, 3, 8, 27, 27], [27, 27, 27, 8, 27, 27], [1, 1, 27, 8, 5, 14], [1, 1, 27, 8, 5, 27], [27, 27, 27, 8, 5, 27], [27, 27, 27, 8, 27, 27], [1, 1, 3, 8, 27, 14], [1, 1, 27, 8, 27, 14], [27, 27, 27, 8, 27, 14], [27, 27, 27, 27, 27, 14], [27, 27, 27, 27, 27, 27], [1, 1, 3, 27, 5, 14], [1, 1, 3, 27, 5, 27], [27, 27, 3, 27, 5, 27], [27, 27, 27, 27, 5, 27], [1, 1, 3, 8, 5, 27], [1, 1, 3, 27, 5, 27], [27, 27, 5, 5], [27, 27, 27, 27], [1, 1, 27, 27], [27, 27, 27, 27], [27, 27, 7], [27, 27, 27], [1, 1, 27], [27, 27, 27], [1, 1, 27, 5, 4], [1, 1, 27, 27, 4], [27, 27, 8, 5, 4], [27, 27, 27, 5, 4], [27, 27, 27, 5, 27], [27, 27, 27, 27, 27], [1, 1, 8, 5, 27], [27, 27, 8, 5, 27], [1, 1, 8, 27, 4], [27, 27, 8, 27, 4], [27, 27, 27, 27, 4], [1, 1, 27, 19], [1, 1, 27, 27], [27, 27, 8, 19], [27, 27, 27, 19], [1, 1, 8, 27], [1, 1, 27, 27], [27, 27, 12], [27, 27, 27], [1, 1, 27],

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8396978 samples, validate on 932998 samples
Epoch 1/10
8396978/8396978 [==============================] - 944s 112us/step - loss: 2.5298 - accuracy: 0.2122 - top_k_categorical_accuracy: 0.6143 - val_loss: 2.3374 - val_accuracy: 0.2557 - val_top_k_categorical_accuracy: 0.6687
Epoch 2/10
8396978/8396978 [==============================] - 941s 112us/step - loss: 2.3222 - accuracy: 0.2593 - top_k_categorical_accuracy: 0.6724 - val_loss: 2.2346 - val_accuracy: 0.2760 - val_top_k_categorical_accuracy: 0.6935
Epoch 3/10
8396978/8396978 [==============================] - 941s 112us/step - loss: 2.2486 - accuracy: 0.2750 - top_k_categorical_accuracy: 0.6898 - val_loss: 2.1820 - val_accuracy: 0.2868 - val_top_k_categorical_accuracy: 0.7057
Epoch 4/10
8396978/8396978 [==============================] - 961s 114us/step - loss: 2.2053 - accuracy: 0.2839 - top_k_categorical_accuracy: 0.7004 - val_loss: 2.1497 - val_accuracy: 0.2933 - val_top_k_categorical_accuracy: 0.7133
Epoch 5/10
8396978/

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
# print("result", api.start_game(practice=1,verbose=True))
# [total_practice_runs,total_recorded_runs,total_recorded_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
# print('run %d practice games out of an allotted 100,000' %total_practice_runs)


## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=True)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

In [ ]:
# win_number = 0
# run_number = 100
# for i in range(run_number):
#     print('Playing ', i, ' th game')
#     # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
#     stat = api.start_game(practice=1,verbose=True)
#     if stat:
#         win_number +=1
#     print("#####################################")
#     print('overall success rate = %.3f' % (win_number/(i+1)))
#     # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
#     time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [8]:
[total_practice_runs,total_recorded_runs,total_recorded_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.861
